In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.1.0
Eager mode:  True
Hub version:  0.8.0
GPU is available


In [3]:
train_data, validation_data, test_data = tfds.load(
    name = "imdb_reviews", 
    split = ('train[:60%]', 'train[60%:]', 'test'),
    as_supervised = True)

Shuffling and writing examples to /home/tanmay/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0DLPIR/imdb_reviews-train.tfrecord


Shuffling and writing examples to /home/tanmay/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0DLPIR/imdb_reviews-test.tfrecord


Shuffling and writing examples to /home/tanmay/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0DLPIR/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /home/tanmay/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

For this example we will use a pre-trained text embedding model from TensorFlow Hub called google/tf2-preview/gnews-swivel-20dim/1.

There are three other pre-trained models to test for the sake of this tutorial:

google/tf2-preview/gnews-swivel-20dim-with-oov/1 - same as google/tf2-preview/gnews-swivel-20dim/1, but with 2.5% vocabulary converted to OOV buckets. This can help if vocabulary of the task and vocabulary of the model don't fully overlap.

google/tf2-preview/nnlm-en-dim50/1 - A much larger model with ~1M vocabulary size and 50 dimensions.

google/tf2-preview/nnlm-en-dim128/1 - Even larger model with ~1M vocabulary size and 128 dimensions.

In [6]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape = [], 
                           dtype = tf.string, trainable = True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [9]:
model.fit(train_data.shuffle(10000).batch(512),
                    epochs = 20,
                    validation_data = validation_data.batch(512),
                    verbose = 1)

Epoch 1/20
30/30 [==============================] - 3s 107ms/step - loss: 0.8705 - accuracy: 0.5293 - val_loss: 0.6971 - val_accuracy: 0.5824
Epoch 2/20
30/30 [==============================] - 2s 82ms/step - loss: 0.6473 - accuracy: 0.5975 - val_loss: 0.6176 - val_accuracy: 0.6222
Epoch 3/20
30/30 [==============================] - 3s 86ms/step - loss: 0.5953 - accuracy: 0.6450 - val_loss: 0.5821 - val_accuracy: 0.6695
Epoch 4/20
30/30 [==============================] - 3s 86ms/step - loss: 0.5570 - accuracy: 0.6833 - val_loss: 0.5423 - val_accuracy: 0.6974
Epoch 5/20
30/30 [==============================] - 3s 84ms/step - loss: 0.5166 - accuracy: 0.7253 - val_loss: 0.5093 - val_accuracy: 0.7271
Epoch 6/20
30/30 [==============================] - 3s 84ms/step - loss: 0.4803 - accuracy: 0.7595 - val_loss: 0.4777 - val_accuracy: 0.7578
Epoch 7/20
30/30 [==============================] - 3s 85ms/step - loss: 0.4431 - accuracy: 0.7871 - val_loss: 0.4484 - val_accuracy: 0.7730
Epoch 8/20
3

In [10]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.311
accuracy: 0.862
